In [1]:
# Modify the file 00_setup.py to define input/output file paths on your system
# The information in 00_setup.py will be used across notebooks
from importlib.machinery import SourceFileLoader
setup = SourceFileLoader("setup", "./00_setup.py").load_module()

# 10: Stellargraph Data Import / Conversion
Convert data into a format suitable for GNN, and import into a heterogeneous graph via stellargraph.  I need to upload data into the graph format, and normalize or rescale the data.  I also will need to handle missing features

I want to have my test set entirely held out (not just the label) and so I create a few stellargraph objects:
  * Training businesses, with their associated NAICS (used for training)
  * Training plus validation businesses, with their associated NAICS (used for model selection / refinement)
  * All businesses (training, validation, test), with their associated NAICS (test nodes used for final evaluation)

*This script takes about 5 minutes on my MacBook Air*

In [2]:
import pandas as pd
import numpy as np

In [3]:
from pathlib import Path
import importlib
import pickle

In [4]:
# Imputer object for easy dataset conversion to GNN friendly format
from sba_gnn.sba_gnn import sg_imputer 
#importlib.reload(sg_imputer)
from sba_gnn.sba_gnn.sg_imputer import GNNImputer

In [5]:
from stellargraph import StellarGraph

2023-11-25 22:26:16.897677: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2023-11-25 22:26:16.897703: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2023-11-25 22:26:16.897709: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2023-11-25 22:26:16.897751: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-11-25 22:26:16.897769: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


## Input training data, descriptives, fit imputer

For a neural network, I need to handle missings and also scale features.  For missing values, I will do a simple median fill for all, but add missing indicators.  

To scale the data, I will do a quantile transform for features with > 5 levels.  This is to avoid scaling binary features.  Then I do a Min/Max scaling on all features, so they are in the [0,1] range

I define a class to do this in sg_imputer.  This class wraps several scikit-learn imputers/scalers so that I can easily fit the objects and then transform the data.  In this section, I show some descriptives to justify my imputation/scaling choices, and also fit the scaler

##### Import train data, show missing

In [6]:
train_df = pd.read_parquet(Path(setup.temp_path).joinpath('01_DATA_combined_base_train.parquet'))

In [7]:
# Examine missingness
pd.concat([train_df[c].isna().value_counts() for c in setup.predictor_features],
         keys=setup.predictor_features)

NoEmp              False    447252
CreateJob          False    447252
LowDoc             False    444025
                   True       3227
DisbursementGross  False    447252
new_business       False    446708
                   True        544
urban_flag         False    358648
                   True      88604
franchise_flag     False    447252
Name: count, dtype: int64

In [8]:
# Same information, more compact
pd.concat([pd.DataFrame([train_df[c].isna().sum()]) for c in setup.predictor_features],
         keys=setup.predictor_features).set_axis(['count_missing'], axis=1)

,,count_missing
NoEmp,0,0
CreateJob,0,0
LowDoc,0,3227
DisbursementGross,0,0
new_business,0,544
urban_flag,0,88604
franchise_flag,0,0


Most fields aren't missing too much.  However I must handle those that are.  I will use a median fill, especially since many of the fields with missingness seem to be binary

##### Explore scaling

In [9]:
# Level counts
feature_level_df = pd.concat([pd.DataFrame([train_df[c].value_counts().count()]) for c in setup.predictor_features],
         keys=setup.predictor_features)
feature_level_df

,,0
NoEmp,0,425
CreateJob,0,182
LowDoc,0,2
DisbursementGross,0,81720
new_business,0,2
urban_flag,0,2
franchise_flag,0,2


A number of features are binary.  I don't need to scale these.  The others must be scaled.  

What I will do is scale features with more than a threshold number of levels using a quantile scaler.  Then I'll use a MinMax scaler on everything to be safe (as a 2 level feature could have values other than 0,1).

##### Create imputer to do the median fill and scaling
This imputer fills missing values, adding missingness indicator features.  Then it quantile scales features with over 5 levels, and MinMax scales the rest.

In [10]:
imputer = GNNImputer(features = setup.predictor_features)                               

In [11]:
train_out = imputer.fit_transform(train_df)

In [12]:
train_out.index

Index([538061, 220915, 237886, 852933, 370109, 825551, 462351, 886865, 687784,
       402987,
       ...
       614020,  30277, 692498, 641806,  12558, 127070, 631061, 821756, 296335,
       521331],
      dtype='int64', name='index', length=447252)

In [13]:
imputer.features_out

['NoEmp',
 'CreateJob',
 'LowDoc',
 'DisbursementGross',
 'new_business',
 'urban_flag',
 'franchise_flag',
 'missingindicator_LowDoc',
 'missingindicator_new_business',
 'missingindicator_urban_flag']

## Transform Test, Validation Data

In [14]:
test_df = pd.read_parquet(Path(setup.temp_path).joinpath('01_DATA_combined_base_test.parquet'))

In [15]:
val_df = pd.read_parquet(Path(setup.temp_path).joinpath('01_DATA_combined_base_val.parquet'))

In [16]:
test_out = imputer.transform(test_df)

In [17]:
val_out = imputer.transform(val_df)

## Combine data
Combine transformed datasets, with key information.  Include the business ID, target status, NAICS features, as well as transformed features from above. Save this data for later use (especially the indices which indicate train/test statustrain_df

In [18]:
train_df.columns

Index(['LoanNr_ChkDgt', 'Name', 'City', 'State', 'Zip', 'Bank', 'BankState',
       'NAICS', 'ApprovalDate', 'ApprovalFY', 'Term', 'NoEmp', 'NewExist',
       'CreateJob', 'RetainedJob', 'FranchiseCode', 'UrbanRural', 'RevLineCr',
       'LowDoc', 'ChgOffDate', 'DisbursementDate', 'DisbursementGross',
       'BalanceGross', 'MIS_Status', 'ChgOffPrinGr', 'GrAppv', 'SBA_Appv',
       'target', 'guaranteed_fract', 'new_business', 'urban_flag',
       'franchise_flag', 'NAICS_2', 'NAICS_sector', 'NAICS_sector_desc',
       'NAICS_num', 'menc_NAICS', 'menc_NAICS_sector',
       'NS___Accommodation and Food Services', 'NS___Construction',
       'NS___Health Care and Social Assistance', 'NS___Manufacturing',
       'NS___Other Services (except Public Administration)',
       'NS___Professional, Scientific, and Technical Services',
       'NS___Retail Trade', 'NS___Wholesale Trade', 'NS___infrequent_sklearn'],
      dtype='object')

In [19]:
naics_features = ['LoanNr_ChkDgt', 'target', 'NAICS', 'NAICS_sector'] +  \
    [c for c in train_df.columns if c.startswith('NS__')]
print(naics_features)

['LoanNr_ChkDgt', 'target', 'NAICS', 'NAICS_sector', 'NS___Accommodation and Food Services', 'NS___Construction', 'NS___Health Care and Social Assistance', 'NS___Manufacturing', 'NS___Other Services (except Public Administration)', 'NS___Professional, Scientific, and Technical Services', 'NS___Retail Trade', 'NS___Wholesale Trade', 'NS___infrequent_sklearn']


In [20]:
comb_naics = pd.concat([train_df[naics_features], test_df[naics_features], 
                        val_df[naics_features]], axis=0, keys=['train', 'test', 'val'])

In [21]:
comb_df = pd.concat([comb_naics,
                     pd.concat([train_out, test_out, val_out], 
                               axis=0, keys=['train', 'test', 'val'])],
                    axis=1) \
    .reset_index(level=0) \
    .rename(columns={'level_0':'dset'}, errors='ignore')
print(comb_df.shape)

(688081, 24)


In [22]:
comb_df

,dset,LoanNr_ChkDgt,target,NAICS,NAICS_sector,NS___Accommodation and Food Services,NS___Construction,NS___Health Care and Social Assistance,NS___Manufacturing,NS___Other Services (except Public Administration),...,NoEmp,CreateJob,LowDoc,DisbursementGross,new_business,urban_flag,franchise_flag,missingindicator_LowDoc,missingindicator_new_business,missingindicator_urban_flag
index,,,,,,,,,,,,,,,,,,,,,
538061,train,5282874009,0,422210,42,0.0,0.0,0.0,0.0,0.0,...,0.887888,0.000000,0.0,0.974845,0.0,1.0,0.0,0.0,0.0,0.0
220915,train,2568556001,0,441222,44-45,0.0,0.0,0.0,0.0,0.0,...,0.519520,0.895395,0.0,0.800167,0.0,0.0,0.0,0.0,0.0,0.0
237886,train,2687465005,1,621310,62,0.0,0.0,1.0,0.0,0.0,...,0.652152,0.691191,0.0,0.107703,0.0,1.0,0.0,0.0,0.0,0.0
852933,train,9174473001,0,448310,44-45,0.0,0.0,0.0,0.0,0.0,...,0.107608,0.000000,1.0,0.433433,0.0,1.0,0.0,0.0,0.0,1.0
370109,train,3598634004,0,541512,54,0.0,0.0,0.0,0.0,0.0,...,0.519520,0.000000,1.0,0.405906,1.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70327,val,1533475000,1,444220,44-45,0.0,0.0,0.0,0.0,0.0,...,0.011119,0.000000,0.0,0.000000,1.0,0.0,0.0,0.0,0.0,0.0
225065,val,2597975001,0,484121,48-49,0.0,0.0,0.0,0.0,0.0,...,0.011119,0.000000,0.0,0.000000,1.0,1.0,0.0,0.0,0.0,0.0
450477,val,4381063007,0,811121,81,0.0,0.0,0.0,0.0,1.0,...,0.011436,0.000000,0.0,0.000000,0.0,1.0,0.0,0.0,0.0,1.0


In [23]:
print(comb_df.columns)

Index(['dset', 'LoanNr_ChkDgt', 'target', 'NAICS', 'NAICS_sector',
       'NS___Accommodation and Food Services', 'NS___Construction',
       'NS___Health Care and Social Assistance', 'NS___Manufacturing',
       'NS___Other Services (except Public Administration)',
       'NS___Professional, Scientific, and Technical Services',
       'NS___Retail Trade', 'NS___Wholesale Trade', 'NS___infrequent_sklearn',
       'NoEmp', 'CreateJob', 'LowDoc', 'DisbursementGross', 'new_business',
       'urban_flag', 'franchise_flag', 'missingindicator_LowDoc',
       'missingindicator_new_business', 'missingindicator_urban_flag'],
      dtype='object')


In [24]:
# TODO FIXME sample combined data for easier testing
#comb_df = comb_df.sample(20000)

In [25]:
comb_df.to_parquet(Path(setup.temp_path).joinpath('10_DATA_combined_scaled_all.parquet'))

## Create Edge Data

##### Functions for creating index for NAICS, NAICS sectors

In [26]:
def get_naics_index(naics_seq):
    return 'n_' + naics_seq

In [27]:
def get_naics_sector_index(naics_sector_seq):
    return 'ns_' + naics_sector_seq.astype('str')

##### Create edge index mapper
Use LoanNr_ChkDgt for the businesses, 'n_' for NAICS codes, and 'ns_' for sectors

In [56]:
# Edges from businesses to NAICS codes
edges_business_naics = comb_df[['LoanNr_ChkDgt', 'NAICS', 'dset']].copy() \
    .rename(columns={'LoanNr_ChkDgt':'source'}) 
edges_business_naics['target'] = get_naics_index(edges_business_naics['NAICS'])
edges_business_naics = edges_business_naics[['source', 'target', 'dset']]
edges_business_naics['type'] = 'loan_naics'

In [57]:
# NAICS to NAICS sectors
edges_naics_sectors = comb_df[['NAICS', 'NAICS_sector', 'dset']].copy() \
    .drop_duplicates() 
edges_naics_sectors['source'] = get_naics_index(edges_naics_sectors['NAICS'])
edges_naics_sectors['target'] = get_naics_sector_index(edges_naics_sectors['NAICS_sector'])
edges_naics_sectors = edges_naics_sectors[['source', 'target', 'dset']]
edges_naics_sectors['type'] = 'naics_sector'

In [58]:
edges_all = pd.concat([edges_business_naics, edges_naics_sectors], axis=0) \
    .reset_index(drop=True)

In [59]:
edges_all.head()

,source,target,dset,type
0,5282874009,n_422210,train,loan_naics
1,2568556001,n_441222,train,loan_naics
2,2687465005,n_621310,train,loan_naics
3,9174473001,n_448310,train,loan_naics
4,3598634004,n_541512,train,loan_naics


In [54]:
edges_all['dset'].value_counts(dropna=False)

dset
train    447252
test     144497
val       96332
Name: count, dtype: int64

In [69]:
edges_all['type'].value_counts(dropna=False)

type
loan_naics      688081
naics_sector      3760
Name: count, dtype: int64

## Get Node Data

In [60]:
# Business node features - these are the original features post scaling
features_business = comb_df[['LoanNr_ChkDgt', 'dset'] + imputer.features_out] \
    .set_index('LoanNr_ChkDgt')

In [61]:
# NAICS features - just the one hots
features_naics = comb_df[['NAICS'] + [c for c in train_df.columns if c.startswith('NS__')]] \
    .drop_duplicates()
features_naics['source'] = get_naics_index(features_naics['NAICS'])
features_naics.set_index('source', inplace=True) 
features_naics.drop(columns='NAICS', inplace=True) 
print(features_naics.shape)

(1311, 9)


In [62]:
# NAICS sector features - also just the one hots
features_naics_sector = comb_df[['NAICS_sector'] + \
                                [c for c in train_df.columns if c.startswith('NS__')]] \
    .drop_duplicates()
features_naics_sector['source'] = get_naics_sector_index(features_naics_sector['NAICS_sector'])
features_naics_sector.set_index('source', inplace=True) 
features_naics_sector.drop(columns='NAICS_sector', inplace=True) 
print(features_naics_sector.shape)

(20, 9)


## Load StellarGraph Objects

##### Function to limit edges, nodes

In [136]:
def limit_data(dsets_list = ['train'],
               edges_all= edges_all, 
               features_business = features_business, 
               features_naics = features_naics, 
               features_naics_sector = features_naics_sector):
    
    # Limit edges
    edges_lim = edges_all[edges_all['dset'].isin(dsets_list)] \
        .drop(columns='dset')
               
    # Business features limit limit
    features_business_lim = features_business[features_business['dset'].isin(dsets_list)] \
        .drop(columns='dset') 
    
    # Get NAICS nodes associated with the dset businesses
    naics_ind = edges_lim[edges_lim['type'] == 'loan_naics'] \
        [['target']] \
        .drop_duplicates()
    
    # Limit NAICS
    features_naics_lim = features_naics.merge(naics_ind.set_index('target'), 
                                              left_index=True, right_index=True)
    
    # Get NAICS sector nodes associated with dset businesses
    naics_sector_ind = edges_lim[edges_lim['type'] == 'naics_sector'] \
        [['target']] \
        .drop_duplicates()
    
    # Limit sectors
    features_naics_sector_lim = features_naics_sector \
        .merge(naics_sector_ind.set_index('target'), left_index=True, right_index=True) 
    
    return (edges_lim, features_business_lim, 
            features_naics_lim, features_naics_sector_lim)

##### Training only

In [138]:
edges_train, features_business_train, features_naics_train, features_naics_sector_train = \
    limit_data(['train'])

In [139]:
print(edges_train.shape)

(448550, 3)


In [140]:
print(f'business features, start rows {features_business.shape[0]}, end {features_business_train.shape[0]}')
print(f'naics features, start rows {features_naics.shape[0]}, end {features_naics_train.shape[0]}')
print(f'naics sector features, start rows {features_naics_sector.shape[0]}, end {features_naics_sector_train.shape[0]}')

business features, start rows 688081, end 447252
naics features, start rows 1311, end 1298
naics sector features, start rows 20, end 20


In [141]:
sba_graph_train = StellarGraph({'LoanNr_ChkDgt':features_business_train,
                                'NAICS': features_naics_train,
                                'NAICS_sector': features_naics_sector_train},
                               edges_train, 
                               source_column="source", target_column="target",
                               edge_type_column="type")

In [151]:
print(sba_graph_train.info())

StellarGraph: Undirected multigraph
 Nodes: 448570, Edges: 448550

 Node types:
  LoanNr_ChkDgt: [447252]
    Features: float32 vector, length 10
    Edge types: LoanNr_ChkDgt-loan_naics->NAICS
  NAICS: [1298]
    Features: float32 vector, length 9
    Edge types: NAICS-loan_naics->LoanNr_ChkDgt, NAICS-naics_sector->NAICS_sector
  NAICS_sector: [20]
    Features: float32 vector, length 9
    Edge types: NAICS_sector-naics_sector->NAICS

 Edge types:
    LoanNr_ChkDgt-loan_naics->NAICS: [447252]
        Weights: all 1 (default)
        Features: none
    NAICS-naics_sector->NAICS_sector: [1298]
        Weights: all 1 (default)
        Features: none


In [142]:
# Save stellargraph object
with open(Path(setup.temp_path).joinpath('10_DATA_stellargraph_train.pkl'), 'wb') as fout:
      pickle.dump(sba_graph_train, fout)

##### Train plus validation

In [143]:
edges_val, features_business_val, features_naics_val, features_naics_sector_val= \
    limit_data(['train','val'])

In [163]:
print(edges_val.shape)

(546091, 3)


In [145]:
print(f'business features, start rows {features_business.shape[0]}, end {features_business_val.shape[0]}')
print(f'naics features, start rows {features_naics.shape[0]}, end {features_naics_val.shape[0]}')
print(f'naics sector features, start rows {features_naics_sector.shape[0]}, end {features_naics_sector_val.shape[0]}')

business features, start rows 688081, end 543584
naics features, start rows 1311, end 1307
naics sector features, start rows 20, end 20


In [146]:
sba_graph_val = StellarGraph({'LoanNr_ChkDgt':features_business_val,
                                'NAICS': features_naics_val,
                                'NAICS_sector': features_naics_sector_val},
                               edges_val, 
                               source_column="source", target_column="target",
                               edge_type_column="type")

In [150]:
print(sba_graph_val.info())

StellarGraph: Undirected multigraph
 Nodes: 544911, Edges: 546091

 Node types:
  LoanNr_ChkDgt: [543584]
    Features: float32 vector, length 10
    Edge types: LoanNr_ChkDgt-loan_naics->NAICS
  NAICS: [1307]
    Features: float32 vector, length 9
    Edge types: NAICS-loan_naics->LoanNr_ChkDgt, NAICS-naics_sector->NAICS_sector
  NAICS_sector: [20]
    Features: float32 vector, length 9
    Edge types: NAICS_sector-naics_sector->NAICS

 Edge types:
    LoanNr_ChkDgt-loan_naics->NAICS: [543584]
        Weights: all 1 (default)
        Features: none
    NAICS-naics_sector->NAICS_sector: [2507]
        Weights: all 1 (default)
        Features: none


In [147]:
# Save stellargraph object
with open(Path(setup.temp_path).joinpath('10_DATA_stellargraph_val.pkl'), 'wb') as fout:
      pickle.dump(sba_graph_val, fout)

##### All nodes (train, validation, test)
No filtering needed

In [152]:
edges_all_fin = edges_all.drop(columns=['dset']) \
    .drop_duplicates()
print(edges_all_fin.shape)

(689392, 3)


In [156]:
features_business_fin = features_business.drop(columns=['dset']) 

In [157]:
sba_graph_all = StellarGraph({'LoanNr_ChkDgt':features_business_fin,
                              'NAICS': features_naics,
                              'NAICS_sector': features_naics_sector},
                             edges_all_fin, 
                             source_column="source", target_column="target",
                             edge_type_column="type")

In [158]:
print(sba_graph_all.info())

StellarGraph: Undirected multigraph
 Nodes: 689412, Edges: 689392

 Node types:
  LoanNr_ChkDgt: [688081]
    Features: float32 vector, length 10
    Edge types: LoanNr_ChkDgt-loan_naics->NAICS
  NAICS: [1311]
    Features: float32 vector, length 9
    Edge types: NAICS-loan_naics->LoanNr_ChkDgt, NAICS-naics_sector->NAICS_sector
  NAICS_sector: [20]
    Features: float32 vector, length 9
    Edge types: NAICS_sector-naics_sector->NAICS

 Edge types:
    LoanNr_ChkDgt-loan_naics->NAICS: [688081]
        Weights: all 1 (default)
        Features: none
    NAICS-naics_sector->NAICS_sector: [1311]
        Weights: all 1 (default)
        Features: none


In [159]:
# Save stellargraph object
with open(Path(setup.temp_path).joinpath('10_DATA_stellargraph.pkl'), 'wb') as fout:
      pickle.dump(sba_graph_all, fout)

##### Test cases *only*
I am interested in how much of the original graph is needed to get good predictions.  I can use a test-only slice to start

In [161]:
edges_test_only, features_business_test_only, \
    features_naics_test_only, features_naics_sector_test_only= \
    limit_data(['test'])

In [162]:
print(edges_test_only.shape)

(145750, 3)


In [164]:
print(f'business features, start rows {features_business.shape[0]}, end {features_business_test_only.shape[0]}')
print(f'naics features, start rows {features_naics.shape[0]}, end {features_naics_test_only.shape[0]}')
print(f'naics sector features, start rows {features_naics_sector.shape[0]}, end {features_naics_sector_test_only.shape[0]}')

business features, start rows 688081, end 144497
naics features, start rows 1311, end 1253
naics sector features, start rows 20, end 20


In [165]:
sba_graph_test_only = StellarGraph({'LoanNr_ChkDgt':features_business_test_only,
                                'NAICS': features_naics_test_only,
                                'NAICS_sector': features_naics_sector_test_only},
                               edges_test_only, 
                               source_column="source", target_column="target",
                               edge_type_column="type")

In [166]:
print(sba_graph_test_only.info())

StellarGraph: Undirected multigraph
 Nodes: 145770, Edges: 145750

 Node types:
  LoanNr_ChkDgt: [144497]
    Features: float32 vector, length 10
    Edge types: LoanNr_ChkDgt-loan_naics->NAICS
  NAICS: [1253]
    Features: float32 vector, length 9
    Edge types: NAICS-loan_naics->LoanNr_ChkDgt, NAICS-naics_sector->NAICS_sector
  NAICS_sector: [20]
    Features: float32 vector, length 9
    Edge types: NAICS_sector-naics_sector->NAICS

 Edge types:
    LoanNr_ChkDgt-loan_naics->NAICS: [144497]
        Weights: all 1 (default)
        Features: none
    NAICS-naics_sector->NAICS_sector: [1253]
        Weights: all 1 (default)
        Features: none


In [167]:
# Save stellargraph object
with open(Path(setup.temp_path).joinpath('10_DATA_stellargraph_test_only.pkl'), 'wb') as fout:
      pickle.dump(sba_graph_test_only, fout)